Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 2: Fluorescence Quantum Yield {-}

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import. 



## Task 1: Emission Spectra of Standard and Unknown {-}

* Visualize the chemical structures of quinine sulfate (CAS 6119-70-6) and fluorescein (CAS 2321-07-5) and comment on why these molecules are fluorophores/dyes.
* Plot the emission and absorption spectra for the standard (quinine) in a single figure. Do a separate plot for then emission and absorption spectra of the unknown (fluorescein). NOTE: the two spectra will probably have different arbitrary units. In order to compare their featuers you may want to normalize the two spectra, e.g. dividing by the intensity of the highest peaks. 
* Report the integrated areas for the spectra of each molecule, and for each concentration of fluoresceine.
* Explicitly report all the other constants and/or known experimental values that you will need to compute the quantum yield at infinite dilution of fluorescein. 

In [ ]:
#@title The Composition of Fluorescein is as follows:
#!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
#!pip install cirpy
import cirpy
quinine=cirpy.resolve('6119-70-6', 'smiles')
fluorescein=cirpy.resolve('2321-07-5', 'smiles')
Draw.MolToImage(Chem.MolFromSmiles(fluorescein))


## Task 2: Quantum Yield of the Standard {-}

The Stern-Volmer equation for self-quenching of solutions allows you to compute the quantum yield of fluorescence from the molar concentration of the dye, $C$, as  

$\frac{\phi^0_s}{\phi_s}=1+14.5\cdot C$

where $\phi^0_s$ is the quantum yield at infinite dilution for the quinine standard ($s$), and the slope of the curve ($14.5\ M^{-1}$) is specific to quinine.

* Use absorbance data at $349.5\ nm$, molar absorption coefficient of quinine (provided in the handsout), cuvette path length of $1\ cm$, and Beers law to find the concentration of quinine in your sample.
* Given that for quinine you know $\phi^0_s=0.545$, use the Stern-Volmer equation to compute the quantum yield of quinine, $\phi_s$, at the concentration used in the experiments. 

## Task 3: Quantum yield of the Unknown {-}

Although the absolute incident intensity of the radiation is unkonwn, the relative incident intensity between an unknown sample and a standard sample will be the same. The quantum yield of the analyte, fluorescein, can thus be determined by comparing the intesity of fluorescence with the one of quinine. 

$\frac{\phi_u}{\phi_s}=\frac{I_u\cdot(1-10^{-A_s})}{I_s\cdot(1-10^{-A_u})}$

where $I_{u/s}$ is the area under the emission spectrum of the unknown/standard (i.e. fluorescein/quinine), while $A_{s/u}$ is the absorbance if the standard/unknown measured at the excitation wavelength. 

* Use the formula above and the results from the previous tasks (see below) to compute the fluorescence quantum yield of fluorescein at the concentrations used in the experiments.

You will need:
* The absorption of quinine $A_s$ at $323\ nm$
* The quantum yeld of quinine from the previous task
* The integrated fluorescence intensity $I_s$ for quinine
* Absorption of fluoresceine $A_u$ at $323\ nm$ for each sample
* Integrated fluorescence intensity $I_u$ for fluoresceine for each sample 

## Task 4: Stern-Volmer Curve for Fluorescein {-}

* Use absorbance data at $485\ nm$, molar absorption coefficient of fluorescein (provided in the handsout), cuvette path length of $1\ cm$, and Beers law to find the concentration of fluoresceins' samples. You may not be able to use Beers law for the high concentration samples, but you can use the dilution factors to determine these concentrations from the low-concentration samples. 
* Plot the inverse of the fluorescence quantum yield of fluorescein (previous task) as a function of concentration. 
* Perform a linear fit of the inverse of the fluorescence quantum yield of fluorescein vs. concentration:
    * From the y-intercept of your linear fit you should obtain the inverse of the fluorescence quantum yield at infinite dilution for fluorescein, $\phi^0_u$.
    * From the slope of your linear fit you should be able to obtain the quenching constant for fluorescein, i.e., the slope of the S-V curve. 


## Discussion {-}

In the final submission, make sure to address the following points:
* Comment on the coupling between the excited state and ground state for both fluorescein and quinine. What measurement allows you to compare the coupling?
* Looking at the molecular structures, can you propose a reasonable explanation for the difference in the infinite dilution fluorescence quantum yields for quinine and fluorescein? Which one couples more strongly to the solvent?
* Which molecule shows a stronger ability to self-quench?  What part of the Stern-Volmer plots provide this information? Can you provide an explanation for the difference between the two molecules?

In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
#!pip install pypandoc

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
#!jupyter nbconvert --no-input --to PDF "/content/drive/MyDrive/Colab Notebooks/QuantumYield_Worksheet.ipynb"